# Getting distinct places and geocoding in Python #

In this script, we're going to start with the .csv file with our regularized place names that we downloaded from OpenRefine and end up with a new .csv file with the same information, but with an additional column for the latitidue and longitude coordinates.

There are a few different steps here:
* Read the .csv file line by line and get the value of the regularized_place_name cell
* Determine whether we've already seen that place. If we have, pass it by. If we haven't, save it.
* Use our list of distinct place names to construct a series of queries to submit via the geonames.org API
* Interpret the results we get from geonames.org to find the latitude and longitude
* Store the place names and corresponding latitude and longitude coordinates
* Copy the information from our original .csv to a new .csv file, but append the appropriate latitude/longitude coordinates for each row.

## Libraries we'll need ##

We'll use a couple of new libraries for this exercise: 
* `csv` reads and writes data from and to .csv files
* `requests` provides easy methods for performing tasks related to getting and sending information over the internet. (The developers bill it as "http for humans.") The requests package isn't the only way to get these things done, but it's easy to work with

We'll also import BeautifulSoup (which we used to parse XML files earlier) and json (which allows us to work with data in JSON--JavaScript Object Notation--format). In practice, we really only need one or the other of these, but I've included examples of both for purposes of illustration, since both XML and JSON are commonly used formats for data interchange. 

(Note: If you're using the RBSDigitalApproaches virtual machine, you'll need to make sure tha you are running this Jupyter notebook with the web-scraping virtualenvironment activated. Use `workon web-scraping` in a terminal and then run `jupyter notebook`.) 

In [1]:
import csv
import requests
from bs4 import BeautifulSoup
import json

## Building a list of distinct place names ##

The .csv file we downloaded from OpenRefine has regularized place names for each record. We could actually just begin geocoding straight away: reading our .csv file, getting the content of the `regularized_place_name` column and sending off a query to geonames.org. If we did that, though, we'd be sending off a *lot* of redundant requests. Our .csv file has 380 rows, but as we saw in OpenRefine, only 25 distinct regularized place names. Think of how many of those rows all read "London UK," for instance. There's no need to ask Geonames the same question over and over again ("Where is London UK?" "Where is London UK?" "Where is London UK?") when the answer's always going to be the same. It would be like living with a four year-old. 

So, instead, we'll begin by reducing our 380 queries to the 25 distinct queries we really have. On the one hand, this is simply the polite thing to do: we're not hitting the Geonames server more than we need to. On the other hand, this can be in our self-interest, as well. Geonames limits the number of requests it will process from a given account per day and per hour, so reducing the number of requests we make keeps us within our limits. 

Now, as it happens, the limit is 30,000 requests in a day and 2,000 in an hour, so we'd be in no danger of hitting that with just 380 requests (though, actually, we *would* exceed our 2,000 hourly requests if all the people in the class all tried to make the same 380 queries simultaneously using the same geonames account). But it's still prudent to limit the number of requests--the fewer requests we make, the fewer chances there are for a response to go astray. Note, too, that, while Geonames is free, you might find yourself needing to use an API that charges, say, per 1,000 requests or something.

And, anyway, we can get a distinct list of place names in just a few lines of code.

In [2]:
# Create an empty list to hold the distinct values from our regularized_place_name column
place_names = []

# Open our .csv file--which you exported from MarcEdit and saved here
with open('/media/sf_RBSDigitalApproaches/output/Thomson_from_ESTC-full_regularized_places.csv', 'r') as infile :
    # Hand our .csv file off to the csv module. We indicate the file we want to read; the delimiter that separates 
    # fields in the file (in this case a comma, but it could be a tab, a space, or anything else--this is an 
    # all-purpose delimited file reader); and the character that's used to set off fields that might happen
    # to contain the delimiter (otherwise, we'd be splitting fields on every comma we encountered, which 
    # could have disastrous effects in the case of something like "by James Thomson. With his last corrections, 
    # additions, and improvements."
    #
    # The csv module's DictReader function deals with .csv data as dictionaries rather than lists: it uses 
    # the fieldname from the first row as the key, and the content of the corresponding cell in a row as the
    # value. This allows us to work with named fields, rather than having to keep track of indices (like
    # remembering that the value we want is in the fifth column of the .csv and looking for row[4].)
    reader = csv.DictReader(infile, delimiter=',',quotechar='"')
    
    # Have our csv DictReader work through the .csv file one line at a time.
    for row in reader :
        # Get the content of the regularized_place_name column.
        reg_place = row['regularized_place_name']
        
        # Check to see if that place name is in the list of distinct place names we're building.
        if reg_place not in place_names :
            
            # If the place name isn't already in our list of distinct place names, add it to our list.
            place_names.append(reg_place)

# Sort our list of distinct place names in alphabetical order. (There is no real need to do this. But it was
# driving me nuts not having it in alphabetical order when I printed it out in the next step for
# illustration purposes...)
place_names = sorted(place_names)

# Let's have a look at the list of distinct place names we've built. Based on what we saw in OpenRefine, there
# should be about 25 of them.
print(place_names)

['Aberdeen UK', 'Alnwick UK', 'Basel CH', 'Belfast UK', 'Berwick UK', 'Boston MA', 'Cork IE', 'Dublin IE', 'Dundee UK', 'Edinburgh UK', 'Glasgow UK', 'Halle DE', 'Hamburg DE', 'Ipswich UK', 'Leipzig DE', 'London UK', 'Newburyport MA', 'Newcastle UK', 'Paris FR', 'Parma IT', 'Perth UK', 'Philadelphia', 'Preston UK', 'The Hague NL', 'Weissenfels DE', 'Wrentham MA']


## Querying Geonames, interpreting the response, and storing the lat/long value (XML version) ##

Now that we have a list of distinct places, we can ask Geonames for their coordinates. We'll use the requests module for building and sending off our queries to api.geonames.org. Then we'll use BeautifulSoup to parse the response (which we'll be getting in XML format) and find the latitude and longitude. Finally, we'll add the distinct place name and its corresponding coordinates to a dictionary so we can retrieve the coordinates later.

In [3]:
# Create an empty dictionary to hold distinct place names as keys and lat/long coordinates as values
geocoded = {}

# Iterate through the items in our place_names list
for place in place_names :
    
    # Define a list of arguments/parameters to send to Geonames, as documented at: 
    # http://www.geonames.org/export/geonames-search.html
    # We'll be searching for our place name (requests handles the URL encoding for us), and we specify
    # that we want only a single result (living dangerously--but I happen to know these 25 work). Finally, 
    # we supply a username. I set this one up for this course. If you are using this example later, please
    # set up your own free account at: http://www.geonames.org/login
    query = [('q', place), ('maxRows', 1), ('username', 'RBSDigitalApproaches')]
    
    # Construct and send our query to geonames with the requests module. Requests combines 
    r = requests.get('http://api.geonames.org/search', params=query)
    print(r.url)
    print(r.text)
    result = r.text
    soup = BeautifulSoup(result, 'xml')
    print(soup('totalResultsCount')[0].string)
    
    if soup('totalResultsCount')[0].string != '0' :
        lat = soup('lat')[0].string
        lng = soup('lng')[0].string
        latlng = lat + ',' + lng
        geocoded.setdefault(place,'')
        geocoded[place] = latlng
    else :
        geocoded[place] = 'error'


http://api.geonames.org/search?q=Aberdeen+UK&maxRows=1&username=RBSDigitalApproaches
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<geonames style="MEDIUM">
<totalResultsCount>890</totalResultsCount>
<geoname>
<toponymName>Aberdeen</toponymName>
<name>Aberdeen</name>
<lat>57.14369</lat>
<lng>-2.09814</lng>
<geonameId>2657832</geonameId>
<countryCode>GB</countryCode>
<countryName>United Kingdom</countryName>
<fcl>P</fcl>
<fcode>PPLA2</fcode>
</geoname>
</geonames>

890
http://api.geonames.org/search?q=Alnwick+UK&maxRows=1&username=RBSDigitalApproaches
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<geonames style="MEDIUM">
<totalResultsCount>6</totalResultsCount>
<geoname>
<toponymName>Alnwick</toponymName>
<name>Alnwick</name>
<lat>55.41318</lat>
<lng>-1.70563</lng>
<geonameId>2657438</geonameId>
<countryCode>GB</countryCode>
<countryName>United Kingdom</countryName>
<fcl>P</fcl>
<fcode>PPL</fcode>
</geoname>
</geonames>

6
http://api.geonames.org/search?q=Basel+CH&max

http://api.geonames.org/search?q=Paris+FR&maxRows=1&username=RBSDigitalApproaches
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<geonames style="MEDIUM">
<totalResultsCount>3371</totalResultsCount>
<geoname>
<toponymName>Paris</toponymName>
<name>Paris</name>
<lat>48.85341</lat>
<lng>2.3488</lng>
<geonameId>2988507</geonameId>
<countryCode>FR</countryCode>
<countryName>France</countryName>
<fcl>P</fcl>
<fcode>PPLC</fcode>
</geoname>
</geonames>

3371
http://api.geonames.org/search?q=Parma+IT&maxRows=1&username=RBSDigitalApproaches
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<geonames style="MEDIUM">
<totalResultsCount>1255</totalResultsCount>
<geoname>
<toponymName>Parma</toponymName>
<name>Parma</name>
<lat>44.79935</lat>
<lng>10.32618</lng>
<geonameId>3171457</geonameId>
<countryCode>IT</countryCode>
<countryName>Italy</countryName>
<fcl>P</fcl>
<fcode>PPLA2</fcode>
</geoname>
</geonames>

1255
http://api.geonames.org/search?q=Perth+UK&maxRows=1&username=RBSDigita

### Querying Geonames and interpreting the response (JSON version) ###

This is the same query we performed in the previous cell, but shows how to use the json module to parse data you receive as JSON. I haven't added it to our geocoded dictionary, because we already have the information from our previous query.

In [4]:
for place in place_names :
    query = [('q', place), ('maxRows', 1), ('username', 'RBSDigitalApproaches')]
    r = requests.get('http://api.geonames.org/searchJSON', params=query)
    # Hand off the text content of the response from requests to the json module
    jsonread = json.loads(r.text)
    # I've included this print command just so you can see what JSON looks like--more like a Python dictionary
    # than an XML document tree
    print(json.dumps(jsonread, indent=4, sort_keys=False))
    # Get the latitude and longitude, in a way analogous to what we did in XML
    lat = jsonread['geonames'][0]['lat']
    lng = jsonread['geonames'][0]['lng']
    # Combine them into a latlng pair
    latlng = lat + ',' + lng
    print('LatLng: ' + latlng)  

{
    "totalResultsCount": 890, 
    "geonames": [
        {
            "fcode": "PPLA2", 
            "countryId": "2635167", 
            "name": "Aberdeen", 
            "countryCode": "GB", 
            "geonameId": 2657832, 
            "toponymName": "Aberdeen", 
            "fclName": "city, village,...", 
            "fcodeName": "seat of a second-order administrative division", 
            "countryName": "United Kingdom", 
            "adminCodes1": {
                "ISO3166_2": "SCT"
            }, 
            "lat": "57.14369", 
            "lng": "-2.09814", 
            "adminName1": "Scotland", 
            "fcl": "P", 
            "adminCode1": "SCT", 
            "population": 196670
        }
    ]
}
LatLng: 57.14369,-2.09814
{
    "totalResultsCount": 6, 
    "geonames": [
        {
            "fcode": "PPL", 
            "countryId": "2635167", 
            "name": "Alnwick", 
            "countryCode": "GB", 
            "geonameId": 2657438, 
            "topo

{
    "totalResultsCount": 895, 
    "geonames": [
        {
            "fcode": "PPL", 
            "countryId": "2921044", 
            "name": "Halle", 
            "countryCode": "DE", 
            "geonameId": 2911522, 
            "toponymName": "Halle (Saale)", 
            "fclName": "city, village,...", 
            "fcodeName": "populated place", 
            "countryName": "Germany", 
            "adminCodes1": {
                "ISO3166_2": "ST"
            }, 
            "lat": "51.48158", 
            "lng": "11.97947", 
            "adminName1": "Saxony-Anhalt", 
            "fcl": "P", 
            "adminCode1": "14", 
            "population": 234107
        }
    ]
}
LatLng: 51.48158,11.97947
{
    "totalResultsCount": 1412, 
    "geonames": [
        {
            "fcode": "PPLA", 
            "countryId": "2921044", 
            "name": "Hamburg", 
            "countryCode": "DE", 
            "geonameId": 2911298, 
            "toponymName": "Hamburg", 
         

{
    "totalResultsCount": 118, 
    "geonames": [
        {
            "fcode": "PPLA2", 
            "countryId": "2635167", 
            "name": "Preston", 
            "countryCode": "GB", 
            "geonameId": 2639912, 
            "toponymName": "Preston", 
            "fclName": "city, village,...", 
            "fcodeName": "seat of a second-order administrative division", 
            "countryName": "United Kingdom", 
            "adminCodes1": {
                "ISO3166_2": "ENG"
            }, 
            "lat": "53.76282", 
            "lng": "-2.70452", 
            "adminName1": "England", 
            "fcl": "P", 
            "adminCode1": "ENG", 
            "population": 313332
        }
    ]
}
LatLng: 53.76282,-2.70452
{
    "totalResultsCount": 87, 
    "geonames": [
        {
            "fcode": "PPLG", 
            "countryId": "2750405", 
            "name": "The Hague", 
            "countryCode": "NL", 
            "geonameId": 2747373, 
            "top

### Progress check ###

Let's see the contents of our dictionary of regularized place names and lat/long pairs

In [5]:
for place, coords in sorted(geocoded.items()) :
    print(place + ': ' + coords)

Aberdeen UK: 57.14369,-2.09814
Alnwick UK: 55.41318,-1.70563
Basel CH: 47.55839,7.57327
Belfast UK: 54.59682,-5.92541
Berwick UK: 55.76536,-2.01186
Boston MA: 42.35843,-71.05977
Cork IE: 51.89797,-8.47061
Dublin IE: 53.33306,-6.24889
Dundee UK: 56.46913,-2.97489
Edinburgh UK: 55.95206,-3.19648
Glasgow UK: 55.86515,-4.25763
Halle DE: 51.48158,11.97947
Hamburg DE: 53.57532,10.01534
Ipswich UK: 52.05917,1.15545
Leipzig DE: 51.33962,12.37129
London UK: 51.50853,-0.12574
Newburyport MA: 42.81259,-70.87728
Newcastle UK: 54.97328,-1.61396
Paris FR: 48.85341,2.3488
Parma IT: 44.79935,10.32618
Perth UK: 56.39522,-3.43139
Philadelphia: 39.95233,-75.16379
Preston UK: 53.76282,-2.70452
The Hague NL: 52.07667,4.29861
Weissenfels DE: 51.20148,11.96843
Wrentham MA: 42.06677,-71.32811


### Write our results to file ###

Now we'll create a new .csv file that keeps all of the bibliographic information from the .csv file we downloaded from OpenRefine and adds a new column with our lat/long pairs

In [6]:
# Open our two .csv files, one in read mode, the other in write mode.
with open('/media/sf_RBSDigitalApproaches/output/Thomson_from_ESTC-full_regularized_places.csv', 'r') as infile, \
open('/media/sf_RBSDigitalApproaches/output/Thomson_from_ESTC_regularized_places_geocoded.csv', 'w') as outfile :
    # Initiate our csv DictReader to read the contents of the file we started with
    reader = csv.DictReader(infile, delimiter=',', quotechar='"')
    
    # Get the fieldnames from our existing .csv
    fieldnames = sorted(reader.next().keys())
    # Add a new entry for 'latlng' to the list of fieldnames
    fieldnames.append('latlng')
    
    # Initiate a csv DictWriter for the new .csv file we're producing
    writer = csv.DictWriter(outfile, delimiter=',',fieldnames=fieldnames)
    # Write the first row of fieldnames
    writer.writeheader()
    for row in reader :
        # Create a new variable to hold the value that we retrieve from the geocoded dictionary, using as our 
        # key the value of the 'regularized_place_name' cell in this row of the .csv file
        coords = geocoded[row['regularized_place_name']]
        # Copy the current row from the .csv file in its entirety (i.e., all the bibliographic information), as
        # a dictionary
        result = row
        # Create a new entry in that dictionary, with key 'latlng' (corresponding to our new fieldname from line
        # 10) and value coords (from line 19).
        result.setdefault('latlng',coords)
        # Write the row to the new .csv file
        writer.writerow(result)
print('File complete.')

File complete.


### Write our results in a different way ###
While we're at it, let's save a different .csv file that groups records together by their place of publication. This .csv can be uploaded to a map layer at Google Maps for a quick-and-dirty display of some of our geocoding work. 

In [9]:
#import csv
# Create an empty dictionary to hold information about our places
data = {}

# Open our .csv file of geocoded records and initiate a csv DictReader to read the fields in each row
with open('/media/sf_RBSDigitalApproaches/output/Thomson_from_ESTC-full_regularized_places.csv', 'r') as infile :
    reader = csv.DictReader(infile, delimiter=',', quotechar='"')
    
    # Read the file one line at a time.
    for row in reader :
        # Get the regularized place name from the row
        place_name = row['regularized_place_name']
        # Use that place name as the key for an entry in the data dictionary. If that key isn't already
        # present, create an empty entry with the place name as the key, and a dictionary as the value.
        # That dictionary consists of two nested keys: 'latlng', which will hold the latitude and longitude
        # cordinates; and 'content', which we start as an empty list for holding information about the books.
        # If there's already an entry for this place name, this code will be ignored.
        data.setdefault(
            place_name,
                {'latlng': '',
                'content': []
                }
        )
        # If we don't already have the latitude and longitude for this place, add it now. 
        if data[place_name]['latlng'] == '' :
            data[place_name]['latlng'] = row['coords']
        # Add to the value of the 'content' key in the nested dictionary for this place name. This line
        # concatenates a string that will include : a permalink to the ESTC, a colon and a space, 
        # the main title of the book, and its publication year in parentheses.
        data[place_name]['content'].append(
            "http://estc.bl.uk/" + row['001'] + ": " + row['245$a'] + " (" + row['008$07:4'] + ")"
        )

# Create a list of fieldnames for the .csv we're about to create
fieldnames = ['latlng','label','content']

# Create a .csv file to hold our output, initiate a csv DictWriter, and write the header row
with open('/media/sf_RBSDigitalApproaches/output/mapping_Thomson.csv', 'w') as outfile :
    writer = csv.DictWriter(outfile, delimiter=',', fieldnames=fieldnames)
    writer.writeheader()
    
    # Get information about each of the places in our data dictionary
    for place_name in data.keys() :
        # Create a dictionary to hold the output we'll write to the .csv file, using our fieldnames
        # as the keys: the latitude/longitude coordinates; the regularized place name; and an empty 
        # entry for the information about the books.
        result = {'latlng':data[place_name]['latlng'], 'label':place_name, 'content':''}
        # Our Google Maps labels are going to get very unwieldy if we try to put too much information
        # in each one. First, we'll check to see if there are more than ten items in the list we called 'content'.  
        if len(data[place_name]['content']) > 10 :
            # If there are, we'll set the value of the 'content' key of our result dictionary to present a message
            # saying how many results there were, and explaining that what follows are the first ten results. Note 
            # how we have to convert the result of len() to a string in order to concatenate it with our text.
            # To present the first ten results, we use the newline character ('\n') as the string that we'll
            # use to join the first ten entries (i.e., 0-9) in our list of books.
            result['content'] = "There are " + str(len(data[place_name]['content'])) + \
            " entries. Here are the first 10:\n" + '\n'.join(data[place_name]['content'][:9])
        # If there are fewer than ten books, we'll just join them all together, separated by newline characters.
        else :
            result['content'] = '\n'.join(data[place_name]['content'])
        
        # Let's print the result so we can see it (because dictionaries are unsorted, the fields won't come out
        # in the order we might expect, necessarily)...
        print(result)
        # ... and write our result out as a line in the .csv file
        writer.writerow(result)    

KeyError: 'coords'